# Buildling an LLM Agent with LangChain 
## Solutions: Agents


**Content:**

1. [Exercise 1: Explore the agent's output](#1)
2. [Exercise 2 : Build your own agent](#2)
3. [Exercise 3: Invoke as many tools as you can](#3)
4. [Exercise 4 : Optimize the agent prompt](#4)

In [4]:
import sys
import os
current_dir = os.path.dirname(os.path.abspath('.'))
folder_b_path = os.path.join(current_dir, 'helper_functions')
sys.path.append(current_dir)

from helper_functions.keys import OPENAI_KEY
from helper_functions.tools import my_own_wiki_tool, weather_tool, image_tool

from langchain.agents import create_tool_calling_agent # set up the agent
from langchain.agents import AgentExecutor # execute agent
from langchain_openai import ChatOpenAI # call openAI as agent llm
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

---

#### Exercise 1: Explore the agent's output <a id='1'></a>

**TASK:**
In the examples below, read the output to observe how the Agent reasons and decides to use a different tool based on the context.

In [5]:
# Agent

# Load Tools
tools = [my_own_wiki_tool, weather_tool, image_tool]

# Load LLM
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, api_key=OPENAI_KEY)

# Get the prompt to use - you can modify this! With this you let the agent know what its purpose is.
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages
print(type(prompt))

# Define  the agent (load the LLM and the list of tools)
agent = create_tool_calling_agent(llm = llm, tools = tools, prompt = prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("Your agent is ready.")


<class 'langchain_core.prompts.chat.ChatPromptTemplate'>
Your agent is ready.


**SOLUTION:**

In [9]:
question_1 = "Where is Amsterdam?"


print(f"Question 1: {question_1}")
agent_executor.invoke({"input": question_1})

Question 1: Where is Amsterdam?


> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Amsterdam'}`


Page: Amsterdam
Summary: Amsterdam ( AM-stər-dam, UK also  AM-stər-DAM, Dutch: [ˌɑmstərˈdɑm] ; literally, "The Dam on the River Amstel") is the capital and most populated city of the Netherlands. It has a population of 921,402 within the city proper, 1,457,018 in the urban area and 2,480,394 in the metropolitan area. Located in the Dutch province of North Holland, Amsterdam is colloquially referred to as the "Venice of the North", for its large number of canals, now a UNESCO World Heritage Site.
Amsterdam was founded at the mouth of the Amstel River, which was dammed to control flooding. Originally a small fishing village in the 12th century, Amsterdam became a major world port during the Dutch Golden Age of the 17th century, when the Netherlands was an economic powerhouse. Amsterdam was the leading centre for finance and trade, as well as a hub of secular art 

{'input': 'Where is Amsterdam?',
 'output': 'Amsterdam is the capital and most populated city of the Netherlands. It is located in the Dutch province of North Holland. Amsterdam is colloquially referred to as the "Venice of the North" due to its large number of canals, which are now a UNESCO World Heritage Site.'}

In [4]:
question_2 = "What is the current temperature in Amsterdam?"

print(f"Question 1: {question_2}")
agent_executor.invoke({"input": question_2})

Question 1: What is the current temperature in Amsterdam?


> Entering new AgentExecutor chain...

Invoking: `weather` with `{'city': 'Amsterdam'}`


Current temperature in Amsterdam: 16.1°CThe current temperature in Amsterdam is 16.1°C.

> Finished chain.


{'input': 'What is the current temperature in Amsterdam?',
 'output': 'The current temperature in Amsterdam is 16.1°C.'}

In [6]:
question_3 = "What should I visit in Amsterdam? Show me an photo"

print(f"Question 1: {question_3}")
agent_executor.invoke({"input": question_3})

Question 1: What should I visit in Amsterdam? Show me an photo


> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Tourist attractions in Amsterdam'}`


Page: List of tourist attractions in Amsterdam
Summary: Amsterdam, one of Europe's capitals, has many attractions for visitors. The city's most famous sight is the 17th-century canals of Amsterdam (in Dutch: grachtengordel), located in the heart of Amsterdam, have been added to the UNESCO World Heritage List.
Invoking: `create_image` with `{'payload': '17th-century canals of Amsterdam'}`


images/image_17th-century_canals_of_Amsterdam.jpg Amsterdam is known for its 17th-century canals, which are a UNESCO World Heritage site. Here is a photo of the canals for you to see: 

![17th-century canals of Amsterdam](images/image_17th-century_canals_of_Amsterdam.jpg)

> Finished chain.


{'input': 'What should I visit in Amsterdam? Show me an photo',
 'output': 'Amsterdam is known for its 17th-century canals, which are a UNESCO World Heritage site. Here is a photo of the canals for you to see: \n\n![17th-century canals of Amsterdam](images/image_17th-century_canals_of_Amsterdam.jpg)'}

---

#### Exercise 2 : Build your own agent  <a id='2'></a>
The goal is to build an agent that uses the tools you previously developed. Feel free to also use the pre-made tools, available at `helper_functions/tools.py`

**TASK**: 
- A template for defining an agent and an API key are already provided to you. Build an agent by using a list of tools, and the LLM `gpt-3.5-turbo-0125`. A template for this agent follows below.
- Test if the agent gives an output.
- Observe how the output changes if you provide less tools in your list of tools.

**SOLUTION:**

In [16]:
# Component 1 (Tools): Load Tools
tools = [
    my_own_wiki_tool, weather_tool, image_tool
]

# Component 2 (LLM): Load LLM
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", 
                 temperature=0, 
                 api_key=OPENAI_KEY)

# Component 3 (Prompt): Let the agent know what its purpose is. For now, let's keep it as is.
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages
print(type(prompt))

# Define the agent and agent executor (load the LLM, the list of tools, and the prompt (descripiton))

agent = create_tool_calling_agent(
    llm = llm, tools = tools, prompt = prompt
    )
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True
)

print("Your agent is ready.")


<class 'langchain_core.prompts.chat.ChatPromptTemplate'>
Your agent is ready.


---

#### Exercise 3: Invoke as many tools as you can  <a id='3'></a>

**TASK:**
Try various questions to call the agent and follow the generated reasoning process in the response. The goal is to call the agent in a way thay it will use as many tools as possible. **Let's see who can reach the highest number of tools used with a single prompt!**

**SOLUTION:** (invoking 2 tools: Wiki and Weather. Can you reach more?)

In [17]:
question = """
Where is Amsterdam and what is the weather like?
"""

print(f"Question: {question}")
agent_executor.invoke({"input": question})

Question: 
Where is Amsterdam and what is the weather like?



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Amsterdam'}`


Page: Amsterdam
Summary: Amsterdam ( AM-stər-dam, UK also  AM-stər-DAM, Dutch: [ˌɑmstərˈdɑm] ; literally, "The Dam on the River Amstel") is the capital and most populated city of the Netherlands. It has a population of 921,402 within the city proper, 1,457,018 in the urban area and 2,480,394 in the metropolitan area. Located in the Dutch province of North Holland, Amsterdam is colloquially referred to as the "Venice of the North", for its large number of canals, now a UNESCO World Heritage Site.
Amsterdam was founded at the mouth of the Amstel River, which was dammed to control flooding. Originally a small fishing village in the 12th century, Amsterdam became a major world port during the Dutch Golden Age of the 17th century, when the Netherlands was an economic powerhouse. Amsterdam was the leading centre for finance and trade, as 

{'input': '\nWhere is Amsterdam and what is the weather like?\n',
 'output': 'Amsterdam is the capital and most populated city of the Netherlands. It is located in the Dutch province of North Holland and is colloquially referred to as the "Venice of the North" due to its large number of canals, which are now a UNESCO World Heritage Site. Amsterdam has a population of 921,402 within the city proper, 1,457,018 in the urban area, and 2,480,394 in the metropolitan area.\n\nSome of the main attractions in Amsterdam include historic canals, the Rijksmuseum, the Van Gogh Museum, Dam Square, the Royal Palace of Amsterdam, the Anne Frank House, and the red-light district. The city is known for its nightlife, festival activity, and artistic heritage.\n\nThe current temperature in Amsterdam is 16.1°C.'}

---

#### Exercise 4 : Optimize the agent prompt  <a id='4'></a>
So far we played around with the provided LLM and list of tools. Now let's look into the 3rd component: the Agent prompt. 

**TASK:**
- Modify the agent prompt and observe the difference in the output. 

**SOLUTION:**

In this solution we assume you are building an agent for a travel agency, and you want the agent to help you motivate people to visit a city.

In [18]:
# Component 1 (Tools): Load Tools from Exercise 1
tools = tools 

# Component 2 (LLM): Load LLM form Exercise 1
llm = llm

your_prompt = """"
    You are a helpful assisstant, trying to motivate people to go on holiday.
"""

# Component 3 (Prompt): Create your own prompt to instruct the Agent about its purpose.
prompt = ChatPromptTemplate.from_messages([
    ("system", your_prompt),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])
prompt.messages


# Define the agent and agent executor (load the LLM, the list of tools, and the prompt (descripiton))
# This is same as in Exercise 1
agent = create_tool_calling_agent(
    llm = llm, tools = tools, prompt = prompt
    )
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True
)

print("Your agent is ready.")


Your agent is ready.


In [19]:
# Observe how the same question from before is answered differently with the different prompt.
print(f"Question: {question}")
agent_executor.invoke({"input": question})

Question: 
Where is Amsterdam and what is the weather like?



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Amsterdam'}`


Page: Amsterdam
Summary: Amsterdam ( AM-stər-dam, UK also  AM-stər-DAM, Dutch: [ˌɑmstərˈdɑm] ; literally, "The Dam on the River Amstel") is the capital and most populated city of the Netherlands. It has a population of 921,402 within the city proper, 1,457,018 in the urban area and 2,480,394 in the metropolitan area. Located in the Dutch province of North Holland, Amsterdam is colloquially referred to as the "Venice of the North", for its large number of canals, now a UNESCO World Heritage Site.
Amsterdam was founded at the mouth of the Amstel River, which was dammed to control flooding. Originally a small fishing village in the 12th century, Amsterdam became a major world port during the Dutch Golden Age of the 17th century, when the Netherlands was an economic powerhouse. Amsterdam was the leading centre for finance and trade, as 

{'input': '\nWhere is Amsterdam and what is the weather like?\n',
 'output': 'Amsterdam is the capital and most populated city of the Netherlands. It is located in the Dutch province of North Holland and is colloquially referred to as the "Venice of the North" due to its large number of canals, which are now a UNESCO World Heritage Site. Amsterdam has a rich history dating back to the 12th century and was a major world port during the Dutch Golden Age of the 17th century.\n\nSome of the main attractions in Amsterdam include historic canals, the Rijksmuseum, the Van Gogh Museum, Dam Square, the Anne Frank House, and the red-light district. The city is known for its nightlife, festivals, and cultural heritage.\n\nThe current temperature in Amsterdam is 16.1°C. It\'s a great time to visit and explore this vibrant city!'}

---